In [1]:
import pymysql.cursors
import pandas as pd
import numpy as np
import csv
import re

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='ta',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Read a single record
        cursor.execute("SELECT review FROM review")
        data_reviews = cursor.fetchall()
        data_review = []
        
        
        for row in data_reviews:
            data_review.append(row['review'])
        reviewdf = pd.DataFrame(np.array(data_review))
        reviewdf.head(5)
        
        reviewdf.to_csv('tes.csv', encoding='utf-8', index = False)
        
finally:
    connection.close()

ModuleNotFoundError: No module named 'pymysql'

In [11]:
#start processing data twitter
def processTweet(tweet):
    # process the tweets
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    print(tweet)
    print(' ')
    return tweet
#end

In [12]:
#initialize stopWords
stopWords = []

In [13]:
#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)
#end

In [14]:
#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords
#end

In [15]:
#start getfeatureVector
def getFeatureVector(tweet):
    featureVector = []
    # tweet=stemmer.stem(tweet)
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stopWords or val is None):
            continue
        else:
           featureVector.append(w.lower())
#     print(featureVector)
#     print(' ')
    return featureVector
#end

In [16]:
#start replace singkatan/kata alay
def translator(tweet):
    j = 0
    for kata in tweet:
        with open('data/singkatan.txt', 'r') as csvfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(csvfile, delimiter="=")
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if kata == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    tweet[j] = row[1]
            csvfile.close()
        j = j + 1
#     print(' '.join(tweet))
#     print('')
    return tweet    
#end process

In [17]:
#Read the tweets one by one and process it
fp = open('tes.csv', 'r')
line = fp.readline()
st = open('data/feature_list/konjungsi.csv', 'r')
stopWords = getStopWordList('data/feature_list/konjungsi.csv')
result_list = []

In [18]:
while line:
    processedTweet = processTweet(line)
#     featureVector = getFeatureVector(processedTweet)
#     replaceAbb = translator(featureVector)
#     result_list.append(replaceAbb)
    line = fp.readline()
#end loop

0 
 
# anakbangsabisa thank you @ gojekindonesia karenamu aku kemana-mana gausah pusing nyari angkot :') 
 
kael ini anak kecil tapi betapa dia terpesona & terinspirasi dgn pelayanan serta keramahan @ gojekindonesia semoga semakin menginspirasi indonesia yaa pic.twitter.com/fqcd3nqvgl 
 
masih banyak orang baik yah ternyata, bayangin kalo itu dompet jatuh ke tangan yg salah... ku suka banget bapak driver @ gojekindonesia satu ini! pic.twitter.com/dyhml9nn65" 
 
pada tanggal 9 agustus 2018, go-jek telah menyerahkan bantuan untuk masyarakat terdampak gempa lombok. bersama dengan pmi, mitra go-jek di lombok ikut turut membantu proses pendistribusian bantuan ke lokasi terdampak di lombok.pic.twitter.com/ri8hgyjgvr" 
 
iya min, semoga pelayanan dan sistem aplikasinya semakin baik ya. saya pengguna setia gojek lho. maju terus untuk gojek!" 
 
ketika lelaki tak bisa diharapkan, @ gojekindonesia selalu jadi andalan." 
 
terima kasih mobile banking @ bni. saya jadi bisa top up gopay @ gojekindo